# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

#Basic DS libs
import numpy as np
import pandas as pd

#Helper Libs
import re
from sqlalchemy import create_engine
import joblib

#NLTK
import nltk
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

#Pipeline
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split

#Vectorizers/Transformers
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

#Models
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV

#Evaluation Metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import hamming_loss, zero_one_loss, jaccard_score, accuracy_score, precision_score, recall_score

#Gensim
from gensim.test.utils import datapath
from gensim import utils
import gensim.models

#Glove
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

#Doc2Vec
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.base import BaseEstimator
import multiprocessing

[nltk_data] Downloading package punkt to /home/tarciso/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/tarciso/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tarciso/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
messages_df = pd.read_sql_table(con=engine, table_name='Message')
categories_df = pd.read_sql_table(con=engine, table_name='MessageCategoryWide')

In [3]:
# create categories_list data with a list of the categories for each message
categories = pd.read_csv('../data/disaster_categories.csv')
categories['categories_list'] = categories['categories'].apply(lambda x: re.sub(r"[_a-z]+-0[;]?", "", x)) \
                                    .apply(lambda x: re.sub(r"-1", "", x)) \
                                    .apply(lambda x: re.sub(r";$", "", x))

In [4]:
categories

,id,categories,categories_list
0,2,related-1;request-0;offer-0;aid_related-0;medi...,related
1,7,related-1;request-0;offer-0;aid_related-1;medi...,related;aid_related;other_aid;weather_related;...
2,8,related-1;request-0;offer-0;aid_related-0;medi...,related
3,9,related-1;request-1;offer-0;aid_related-1;medi...,related;request;aid_related;medical_products;o...
4,12,related-1;request-0;offer-0;aid_related-0;medi...,related
...,...,...,...
26243,30261,related-0;request-0;offer-0;aid_related-0;medi...,
26244,30262,related-0;request-0;offer-0;aid_related-0;medi...,
26245,30263,related-1;request-0;offer-0;aid_related-0;medi...,related
26246,30264,related-1;request-0;offer-0;aid_related-1;medi...,related;aid_related;military


### 2 - Design and Apply Tokenization Strategies

#### 2.1 - Write down tokenization functions

In [5]:
def tokenize_to_list(text, lemmatizer):
    '''
    INPUT
    text - text string to be tokenized
    lemmatizer - lemmatizer object to be used to process text tokens
    
    OUTPUT
    A list of tokens extracted from the input text
    
    This function receives raw text as input a pre-processes it for NLP analysis, removing punctuation and
    special characters, normalizing case and removing extra spaces, as well as removing stop words and 
    applying lemmatization
    '''
    tokens = nltk.tokenize.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip()))
    clean_tokens = [lemmatizer.lemmatize(tok) for tok in tokens if tok not in stopwords.words("english")]

    return clean_tokens

In [6]:
def tokenize_to_str(text, lemmatizer):
    '''
    INPUT
    text - text string to be tokenized
    lemmatizer - lemmatizer object to be used to process text tokens
    
    OUTPUT
    A string with the tokens extracted from the input text concatenated by spaces
    
    This function receives raw text as input a pre-processes it for NLP analysis, removing punctuation and
    special characters, normalizing case and removing extra spaces, as well as removing stop words and 
    applying lemmatization
    '''
    tokens = nltk.tokenize.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip()))
    clean_tokens = [lemmatizer.lemmatize(tok) for tok in tokens if tok not in stopwords.words("english")]
    #Return tokens list as a string joined by whitespaces
    clean_tokens_str = ' '.join(clean_tokens)

    return clean_tokens_str

In [7]:
lemmatizer = WordNetLemmatizer()
tokenize_to_list(messages_df.message[0],lemmatizer)

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

In [8]:
tokenize_to_str(messages_df.message[0],lemmatizer)

'weather update cold front cuba could pas haiti'

#### 2.2 - Generate tokenized messages table for further analysis

In [9]:
# %%time
# lemmatizer = WordNetLemmatizer()
# messages_df['tokens_list'] = messages_df.message.apply(lambda x: tokenize_to_list(x, lemmatizer))

In [10]:
# %%time
# messages_df['tokens_str'] = messages_df.message.apply(lambda x: tokenize_to_str(x, lemmatizer))

In [11]:
messages_df

,message_id,message,original,genre,num_words
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,13
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,9
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,6
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,13
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,12
...,...,...,...,...,...
26139,30261,The training demonstrated how to enhance micro...,None,news,21
26140,30262,A suitable candidate has been selected and OCH...,None,news,22
26141,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,23
26142,30264,"Some 2,000 women protesting against the conduc...",None,news,31


In [12]:
# messages_tokens = messages_df[['message_id','tokens_str']]

In [13]:
# messages_tokens.to_sql('MessageTokens', engine, index=False, if_exists='replace')

In [14]:
#messages_categories = messages_tokens.merge(categories_df, on='message_id')

In [15]:
#messages_categories.head()

#### 2.4 - Find n-grams and save to DB

In [16]:
# def get_ngrams_freqs(messages_array, n=1):
#     vec = CountVectorizer(ngram_range=(n, n)).fit(messages_array)
#     bag_of_words = vec.transform(messages_array)
#     word_count = bag_of_words.sum(axis=0)
#     words_freq = [(word, n, word_count[0, idx]) for word, idx in vec.vocabulary_.items()]
#     words_freq = sorted(words_freq, key = lambda x: x[2], reverse=True)
#     words_freq_df = pd.DataFrame(data = words_freq, columns = ['ngram','n','count'])
#     return words_freq_df

In [17]:
# unigrams_freqs = get_ngrams_freqs(messages_tokens.tokens_str)
# bigrams_freqs = get_ngrams_freqs(messages_tokens.tokens_str, n=2)
# trigrams_freqs = get_ngrams_freqs(messages_tokens.tokens_str, n=3)
# ngrams_freqs = pd.concat([unigrams_freqs, bigrams_freqs, trigrams_freqs])

In [18]:
# ngrams_freqs

In [19]:
# ngrams_freqs.to_sql('NGramsFreqs', engine, index=False, if_exists='replace')

### 3 - Prepare Data for Train/Test Pipeline
- Extract X and Y from datasets
- Split data into train and test sets
- Train pipeline

In [20]:
messages_tokens = pd.read_sql_table(con=engine, table_name='MessageTokens')

In [21]:
messages_tokens

,message_id,tokens_str
0,2,weather update cold front cuba could pas haiti
1,7,hurricane
2,8,looking someone name
3,9,un report leogane 80 90 destroyed hospital st ...
4,12,say west side haiti rest country today tonight
...,...,...
26139,30261,training demonstrated enhance micronutrient pr...
26140,30262,suitable candidate selected ocha jakarta curre...
26141,30263,proshika operating cox bazar municipality 5 un...
26142,30264,2 000 woman protesting conduct election tearga...


In [22]:
# prepare data for ML Pipeline
X = messages_df.message.values
X_tokenized = messages_tokens.tokens_str.values
Y_df = categories_df.drop('message_id', axis=1)
Y = Y_df.values
category_columns = Y_df.columns

In [23]:
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [24]:
X_tokenized

array(['weather update cold front cuba could pas haiti', 'hurricane',
       'looking someone name', ...,
       'proshika operating cox bazar municipality 5 union ramu chokoria assessment 5 kg rice 1 5 kg lentil 700 family',
       '2 000 woman protesting conduct election teargassed tried converge local electoral commission office southern oil city port harcourt',
       'radical shift thinking came result meeting recognizing hiv aid core humanitarian crisis identifying crisis function hiv aid pandemic'],
      dtype=object)

In [25]:
Y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [26]:
category_columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=199)

In [28]:
X_train

array(['crudess you are the most charm and charismatic DT member .. thanks for all your videos in Santiago.. and to talk with me twice',
       'Tidal waves triggered by an earthquake off Indonesia on Sunday swept over a vast swath of coastlines, including those in India, Indonesia, Malaysia, the Maldives, Somalia, Sri Lanka and Thailand.',
       'The earthquake happened at 4h34 in the evening and for the night. ',
       ...,
       'Earthquake of the day #Haiti or Google possibly leaving China ?',
       'When the matter had been debated previously, the Non-Aligned Movement foreign ministers firmly stated that there was no right to humanitarian intervention.',
       '@DumboNYC : Oh no RT @endtwist : ConEd power station in #DUMBO is flooding ! #nopower #sandy'],
      dtype=object)

In [29]:
y_train

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [30]:
print(X_train.shape, y_train.shape)

(19608,) (19608, 36)


___

## Feature Engineering

##### 4 - Design Transformer to Tokenize sentences before passing on to other transformers

In [31]:
class Tokenizer(BaseEstimator):

    def __init__(self, output_str=True):
        self.output_str = output_str
        self.lemmatizer = WordNetLemmatizer()

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([tokenize_to_str(tokens_str, lemmatizer) if self.output_str else 
                         tokenize_to_list(tokens_str, lemmatizer) for tokens_str in X])
            

In [32]:
# %%time
# msgTokenizer = Tokenizer().fit(X, None)
# msgTokenizer.transform(X)

### 4 - Create a Mean/TF-IDF word vector from a locally trained Word2Vec model

#### 4.1 - Train a Word2Vec model using messages text

In [33]:
# class Message(object):
#     """An interator that yields messages tokens (lists of str)."""
    
#     def __init__(self, messages_df, sample_size=-1):
#         self.messages_df = messages_df
#         self.sample_size = sample_size

#     def __iter__(self):
#         messages = messages_df
        
#         #Sample dataset if specified
#         if self.sample_size > 0:
#             messages = messages.sample(self.sample_size)
        
#         #Yeld tokenized message joined by whitespaces
#         for token_str in messages.tokens_str:
#             yield token_str.split()

In [34]:
# messages_df.tokens_str

In [35]:
# %%time

# sentences = Message(messages_df=messages_df)
# local_w2v_model_full = gensim.models.Word2Vec(sentences=sentences)
# local_w2v_model_full.save("messages-word2vec-full.model")

In [36]:
# w2v_model_full = gensim.models.Word2Vec.load("messages-word2vec-full.model")

In [37]:
# # look up generated word vocab
# for i, word in enumerate(w2v_model_full.wv.vocab):
#     if i == 10:
#         break
#     print(word)

In [38]:
# # look up generated word vectors
# w2v_model_full.wv['earthquake']

#### 4.2 - Implement Vector Aggregator

Inspecting variables

In [39]:
# len(X)

In [40]:
# X[0]

In [41]:
# X.shape[0]

Initial implementation of aggregators

In [42]:
from collections import defaultdict

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec_model):
        self.word2vec_model = word2vec_model
        self.num_dims = word2vec_model.vector_size
            
    def fit(self, X, y):
        return self 

    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

In [43]:
# #Testing defaultdict functionality
# test_dict = defaultdict(lambda: 1, [("Hello" , 7), ("hi" , 10), ("there" , 45),("at" , 23),("this" , 77)])
# test_dict['Hi']

In [44]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec_model):
        self.word2vec_model = word2vec_model
        self.num_dims = word2vec_model.vector_size
        self.word_weights = None
        
    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        
        tfidf_weights = [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()]
        self.word_weights = defaultdict(lambda: max_idf, tfidf_weights)
    
        return self
    
    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model[w]*self.word_weights[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

Test aggregators using a Pipeline

In [45]:
# %%time

# w2v_mean_pip = Pipeline([
#     ('features',FeatureUnion([
#         ('w2v_mean', MeanEmbeddingVectorizer(w2v_model_full)),
#         ('w2v_tfidf', TfidfEmbeddingVectorizer(w2v_model_full))
#     ])),    
#     ('clf', MultiOutputClassifier(GaussianNB()))
# ])

# w2v_mean_pip.fit(X_train, y_train)

# y_pred = w2v_mean_pip.predict(X_test)

# for category_idx in range(y_pred.shape[1]):
#     print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))

In [46]:
# category_idx=1
# c_report = classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], 
#                       labels=[0,1], 
#                       target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1'],
#                       output_dict=True)
# c_report

In [47]:
# print(c_report.keys())

In [48]:
# w2v_mean_pip.score(X_test, y_test)

In [49]:
# y_pred[0]

y_pred.shape

y_test[0]

y_test.shape

sum(y_pred[0] == y_test[0])/y_pred[0].shape[0]

sample_test = y_test[0]
sample_pred = y_pred[0]

sample_test == 1

sample_pred == 1

(sample_test == 1) == (sample_pred == 1)

#Computing TPR manually
tp_array = (sample_test == 1) & (sample_pred == 1)
tp = tp_array.sum()
tpr = tp/sample_test.shape[0]
print(tp)
print(tpr)
tp_array

3/36

#Computing FPR manually
fp_array = (sample_test == 0) & (sample_pred == 1)
fp = fp_array.sum()
fpr = fp/sample_test.shape[0]
print(fp)
print(fpr)
fp_array

#Computing FNR manually
fn_array = (sample_test == 1) & (sample_pred == 0)
fn = fn_array.sum()
fnr = fn/sample_test.shape[0]
print(fn)
print(fnr)
fn_array

#Computing Precision manually
print(tp/(tp+fp))

#Computing Recall manually
print(tp/(tp+fn))

example_test = np.append(np.repeat(1, 4),np.repeat(0,32))
example_test

example_pred = np.repeat(0, 36)
example_pred

from sklearn.metrics import f1_score

print("Hamming Loss = ", hamming_loss(example_test,example_pred))
print("Zero-One Loss = ", zero_one_loss(example_test,example_pred))
print("Jaccard Score = ", jaccard_score(example_test,example_pred))
print("Accuracy = ", accuracy_score(example_test,example_pred))
print("Recall = ", recall_score(example_test,example_pred))
print("Precision = ", precision_score(example_test,example_pred))
print("F1-Score = ", f1_score(example_test,example_pred))
print("Jaccard Score Micro = ", jaccard_score(example_test,example_pred, average='micro'))
print("Recall Micro = ", recall_score(example_test,example_pred, average='micro'))
print("Precision Micro = ", precision_score(example_test,example_pred, average='micro'))
print("F1-Score Micro = ", f1_score(example_test,example_pred, average='micro'))

print("Hamming Loss = ", hamming_loss(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Zero-One Loss = ", zero_one_loss(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Jaccard Score = ", jaccard_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))
print("Accuracy = ", accuracy_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Recall = ", recall_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))
print("Precision = ", precision_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))

from sklearn.metrics import mean_absolute_error, r2_score, explained_variance_score
print(mean_absolute_error(y_test, y_pred, multioutput='raw_values'))
print(mean_absolute_error(y_test, y_pred))
(y_test == y_pred).mean()
print(r2_score(y_test, y_pred))
print(r2_score(y_test, y_pred, multioutput='variance_weighted'))
print(explained_variance_score(y_test, y_pred))
print(explained_variance_score(y_test, y_pred, multioutput='variance_weighted'))

#### 4.3 - Create Estimator to both train and average w2v on corpus data

Trying W2VTransformer - Gensim's sklearn Transformer

from gensim.sklearn_api import W2VTransformer
from gensim.test.utils import common_texts

model = W2VTransformer(size=10, min_count=1, seed=1)

model.fit(common_texts)
          
model.transform(['graph', 'system'])

common_texts

messages_df.head(5).tokens_str.str.split().values

messages_df.head(5).tokens_str

w2v_model = W2VTransformer()
w2v_model.fit(messages_df.tokens_str.str.split().values)

word = messages_df.tokens_str[0].split()[1]
print(word)
w2v_model.transform(word)

test_model = gensim.models.Word2Vec(messages_df.tokens_str.str.split().values)

for i, word in enumerate(test_model.wv.vocab):
    if i == 10:
        break
    print(word)

test_model.wv['earthquake']

Re-implement aggregators now with training capability

In [50]:
class MeanEmbeddingTrainVectorizer(object):

    def __init__(self, word2vec_model=None):
        if word2vec_model is None:
            self.word2vec_model = None
            self.num_dims = -1
        else:
            self.word2vec_model = word2vec_model
            self.num_dims = word2vec_model.vector_size
            
    def fit(self, X, y):
        if self.word2vec_model is None:
            self.word2vec_model = gensim.models.Word2Vec(X)
            self.num_dims = self.word2vec_model.vector_size
            
        return self 

    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

In [51]:
class TfidfEmbeddingTrainVectorizer(object):
    
    def __init__(self, word2vec_model=None):
        if word2vec_model is None:
            self.word2vec_model = None
            self.num_dims = -1
        else:
            self.word2vec_model = word2vec_model
            self.num_dims = word2vec_model.vector_size
        self.word_weights = None
        
        
    def fit(self, X, y):
        if self.word2vec_model is None:
            self.word2vec_model = gensim.models.Word2Vec(X)
            self.num_dims = self.word2vec_model.vector_size
            
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        
        tfidf_weights = [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()]
        self.word_weights = defaultdict(lambda: max_idf, tfidf_weights)
    
        return self
    
    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model[w]*self.word_weights[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

Test aggregators using a Pipeline

%%time

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

w2v_mean_pip = Pipeline([
    ('features',FeatureUnion([
        ('w2v_mean', MeanEmbeddingTrainVectorizer(w2v_model_full)),
        ('w2v_mean_train', MeanEmbeddingTrainVectorizer()),
        ('w2v_tfidf', TfidfEmbeddingTrainVectorizer(w2v_model_full)),
        ('w2v_tfidf_train', TfidfEmbeddingTrainVectorizer()),
    ])),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

w2v_mean_pip.fit(X_train, y_train)

y_pred = w2v_mean_pip.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))

### 5. Use Glove pre-trained model to generate feature vectors

Glove pre-trained vectors can be downloaded here: https://nlp.stanford.edu/projects/glove/

%%time

# Code which can be used to convert from Glove default format to Gensim W2V format

glove_file = 'glove-pretrained/glove.6B.50d.txt'
tmp_file = 'glove-pretrained/glove.6B.50d_word2vec.txt'

#_ = glove2word2vec(glove_file, tmp_file)

glove_model = KeyedVectors.load_word2vec_format(tmp_file)

In [52]:
%%time

from gensim.models import KeyedVectors

glove_300d_w2v = KeyedVectors.load_word2vec_format('glove-pretrained/glove.6B.300d_word2vec.txt', binary=False)

CPU times: user 2min 32s, sys: 1.39 s, total: 2min 33s
Wall time: 2min 33s


##### Use a Pipeline to train the Word2Vec estimator

Test Aggregators with Glove using a Pipeline

%%time

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

w2v_mean_pip = Pipeline([
    ('features',FeatureUnion([
        ('glove_mean', MeanEmbeddingTrainVectorizer(glove_300d_w2v)),
        ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
    ])),    
    ('clf', MultiOutputClassifier(GaussianNB()))
])

w2v_mean_pip.fit(X_train, y_train)

y_pred = w2v_mean_pip.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))
    
print("y_pred sum = ",y_pred.sum())
print("Hamming Loss = ", hamming_loss(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Zero-One Loss = ", zero_one_loss(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Jaccard Score = ", jaccard_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))
print("Accuracy = ", accuracy_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))

In [53]:
#from sklearn.metrics import precision_recall_curve
#from sklearn.metrics import average_precision_score

# For each class
# precision = dict()
# recall = dict()
# average_precision = dict()
# for i in range(len(category_columns)):
#     precision[i], recall[i], _ = precision_recall_curve(y_test[:, i], y_pred[:, i])
    #average_precision[i] = average_precision_score(y_test[:, i], y_pred[:, i])

# A "micro-average": quantifying score on all classes jointly
# precision["micro"], recall["micro"], _ = precision_recall_curve(Y_test.ravel(),
#     y_score.ravel())
# average_precision["micro"] = average_precision_score(Y_test, y_score,
#                                                      average="micro")
# print('Average precision score, micro-averaged over all classes: {0:0.2f}'
#       .format(average_precision["micro"]))

from sklearn.metrics import hamming_loss, zero_one_loss, jaccard_score, accuracy_score, auc, recall_score, precision_score

print("Hamming Loss = ", hamming_loss(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Zero-One Loss = ", zero_one_loss(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Jaccard Score = ", jaccard_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))
print("Accuracy = ", accuracy_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Recall = ", recall_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))
print("Precision = ", precision_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))

y_test[0] == y_pred[0]

y_test[0]

y_pred[0]

6536*36

print(np.argmax(y_pred, axis=1).shape)
print(y_pred.shape)

from sklearn.preprocessing import MultiLabelBinarizer
train_foo = [['sci-fi', 'thriller'],['comedy'],['sci-fi', 'thriller'],['comedy']]
mlb = MultiLabelBinarizer()
mlb_label_train = mlb.fit_transform(train_foo)
mlb_label_train

y_test

y_pred

Y_df[Y_df['related'] == 2]

### 6. Use Doc2Vec to generate feature vectors

# Checking the number of words in the corpus
messages_df['message'].apply(lambda x: len(x.split(' '))).sum()

Try Doc2Vec step by step

def read_messages(messages, tokens_only=False):
    for index, tokens_str in np.ndenumerate(messages):
        tokens = tokens_str.split()
        if tokens_only:
            yield tokens
        else:
            # For training data, add tags
            tags = index
            yield gensim.models.doc2vec.TaggedDocument(tokens, tags)

train_corpus = list(read_messages(X_train))
test_corpus = list(read_messages(X_test, tokens_only=True))

train_corpus[:10]

test_corpus[:10]

d2v_model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=5, epochs=10)

d2v_model.build_vocab(train_corpus)

%%time 

d2v_model.train(train_corpus, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

#### Create a Doc2Vec Estimator/Transformer 

In [54]:
class Doc2VecTransformer(BaseEstimator):

    def __init__(self, vector_size=100, epochs=20):
        self.epochs = epochs
        self.vector_size = vector_size
        self.workers = multiprocessing.cpu_count() - 1
        self.d2v_model = None

    def fit(self, X, y):
        tagged_x = [TaggedDocument(tokens_str.split(), [index]) for index, tokens_str in np.ndenumerate(X)]
        self.d2v_model = Doc2Vec(vector_size=self.vector_size, workers=self.workers, epochs=self.epochs)
        self.d2v_model.build_vocab(tagged_x)
        self.d2v_model.train(tagged_x, total_examples=self.d2v_model.corpus_count, 
                             epochs=self.epochs)

        return self

    def transform(self, X):
        return np.asmatrix(np.array([self.d2v_model.infer_vector(tokens_str.split())
                                     for tokens_str in X]))

%%time

doc2vec_trf = Doc2VecTransformer(epochs=1)
doc2vec_trf.fit(X_train, y_train)

X_test[0].split()

inferred_vector = doc2vec_trf.d2v_model.infer_vector(X_test[0].split())
print(inferred_vector.shape)
print(type(inferred_vector))
print(inferred_vector)

inferred_vector2 = np.asmatrix(np.array([doc2vec_trf.d2v_model.infer_vector(X_test[0].split())]))
print(inferred_vector2.shape)
print(type(inferred_vector2))
print(inferred_vector2)

X_train

X_test

doc2vec_features = doc2vec_trf.transform(X_test)
print(doc2vec_features.shape)
doc2vec_features

Test Doc2Vec using a Pipeline

%%time

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

w2v_mean_pip = Pipeline([
    ('doc2vec', Doc2VecTransformer()),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

w2v_mean_pip.fit(X_train, y_train)

y_pred = w2v_mean_pip.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))
    
print("y_pred sum = ",y_pred.sum())

### 7. Extra Feature - Distance Between Message Word Vector to Categories Word Vectors 

We will use Glove as our space vectorizer as it is trained on a broad corpus and thus able to better describe texts / words

First, let's build the embeddings for the categories

In [55]:
category_columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [56]:
categories_tokens = np.array([np.array(cat.split('_')) for cat in category_columns])

In [57]:
categories_tokens

array([array(['related'], dtype='<U7'), array(['request'], dtype='<U7'),
       array(['offer'], dtype='<U5'),
       array(['aid', 'related'], dtype='<U7'),
       array(['medical', 'help'], dtype='<U7'),
       array(['medical', 'products'], dtype='<U8'),
       array(['search', 'and', 'rescue'], dtype='<U6'),
       array(['security'], dtype='<U8'), array(['military'], dtype='<U8'),
       array(['child', 'alone'], dtype='<U5'),
       array(['water'], dtype='<U5'), array(['food'], dtype='<U4'),
       array(['shelter'], dtype='<U7'), array(['clothing'], dtype='<U8'),
       array(['money'], dtype='<U5'),
       array(['missing', 'people'], dtype='<U7'),
       array(['refugees'], dtype='<U8'), array(['death'], dtype='<U5'),
       array(['other', 'aid'], dtype='<U5'),
       array(['infrastructure', 'related'], dtype='<U14'),
       array(['transport'], dtype='<U9'),
       array(['buildings'], dtype='<U9'),
       array(['electricity'], dtype='<U11'),
       array(['tools'], dtype

glove_300d_w2v_mean = MeanEmbeddingTrainVectorizer(glove_300d_w2v)

glove_300d_w2v_mean.fit(categories_tokens, None)

cat_tokens_embeddings = glove_300d_w2v_mean.transform(categories_tokens)

print(cat_tokens_embeddings.shape)
print(cat_tokens_embeddings)

cat_tokens_embeddings[0]

test_msgs_embeddings = glove_300d_w2v_mean.transform(X_test)

print(test_msgs_embeddings.shape)
print(test_msgs_embeddings)

test_msgs_embeddings[0]

from sklearn.metrics.pairwise import cosine_similarity

print(test_msgs_embeddings[:10].shape)
print(cat_tokens_embeddings.shape)
print(cosine_similarity(test_msgs_embeddings[:10],cat_tokens_embeddings).shape)

#### Create a CategoriesSimilarity Transformer 

In [58]:
from sklearn.metrics.pairwise import cosine_similarity

class CategoriesSimilarity(object):
    
    def __init__(self, word2vec_model, categories_tokens):
        self.word2vec_model = word2vec_model
        self.categories_tokens = categories_tokens
        self.categories_vectors = None
        self.num_dims = word2vec_model.vector_size
        
    def compute_mean_embeddings(self, tokens_array):    
        mean_embeddings = np.empty([tokens_array.shape[0],self.num_dims])
        
        for i in range(tokens_array.shape[0]):
            doc_tokens = tokens_array[i]
            
            words_vectors_concat = [self.word2vec_model[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings
                    
    def fit(self, X, y):
        self.categories_vectors = self.compute_mean_embeddings(self.categories_tokens)
        return self 

    def transform(self, X):
        mean_embeddings = self.compute_mean_embeddings(X)
        cats_similarities = cosine_similarity(mean_embeddings, self.categories_vectors)
            
        return cats_similarities

Test the transformer manually using test data

cats_sim_transf = CategoriesSimilarity(glove_300d_w2v, categories_tokens)

cats_sim_transf.fit(X_test, None)

cats_sim_transf.transform(X_test)

Test CategoriesSimilarity feature using a Pipeline

%%time

from sklearn.pipeline import FeatureUnion

cats_sim_pip = Pipeline([
    ('cats_sim', CategoriesSimilarity(glove_300d_w2v, cat_tokens_embeddings)),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

cats_sim_pip.fit(X_train, y_train)

y_pred = cats_sim_pip.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))
    
print("Score = ", cats_sim_pip.score(X_test, y_test))

# accuracy
print("Accuracy = ",accuracy_score(y_test,y_pred))
print("\n")

### 8. NLP Feature Selection using Sklearn Pipelines

#### 8.1 - All features together

%%time

from sklearn.pipeline import FeatureUnion

all_feats = Pipeline([
    ('features', FeatureUnion([
        ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
        ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
        ('doc2vec', Doc2VecTransformer()),
        ('cats_sim', CategoriesSimilarity(glove_300d_w2v, categories_tokens))
    ])),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

all_feats.fit(X_train, y_train)

y_pred = all_feats.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))

score = all_feats.score(X_test, y_test)

score

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize_to_list(lemmatizer=lemmatizer), ngram_range=(1,3))),
    ('tfidf', TfidfTransformer()),
    ('multi_clf', MultiOutputClassifier(RandomForestClassifier(random_state=199), n_jobs=-1))
])

In [59]:
# pipeline.fit(X_train, y_train)

#### Build pipelines for different combinations of features and classifiers

from sklearn import svm

local_w2v_tfidf_svc = Pipeline([
    ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
    ('clf', MultiOutputClassifier(svm.LinearSVC(random_state=199)))
])

local_w2v_tfidf_rf = Pipeline([
    ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

glove_tfidf_svc = Pipeline([
    ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
    ('clf', MultiOutputClassifier(svm.LinearSVC(random_state=199)))
])

glove_tfidf_rf = Pipeline([
    ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

doc2vec_svc = Pipeline([
    ('doc2vec', Doc2VecTransformer()),
    ('clf', MultiOutputClassifier(svm.LinearSVC(random_state=199)))
])

doc2vec_rf = Pipeline([
    ('doc2vec', Doc2VecTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

all_feats_svc = Pipeline([
    ('features', FeatureUnion([
        ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
        ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
        ('doc2vec', Doc2VecTransformer()),
        ('cats_sim', CategoriesSimilarity(glove_300d_w2v, categories_tokens))
    ])),
    ('clf', MultiOutputClassifier(svm.LinearSVC(random_state=199)))
])

all_feats_rf = Pipeline([
    ('features', FeatureUnion([
        ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
        ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
        ('doc2vec', Doc2VecTransformer()),
        ('cats_sim', CategoriesSimilarity(glove_300d_w2v, categories_tokens))
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

pipelines = [local_w2v_tfidf_svc, local_w2v_tfidf_rf, glove_tfidf_svc, glove_tfidf_rf, 
             doc2vec_svc, doc2vec_rf, all_feats_svc, all_feats_rf]

grid_dict = {0 : 'Local Word2Vec - TF-IDF - Linear SVC',
             1 : 'Local Word2Vec - TF-IDF - Random Forest',
             2 : 'Glove 300d - TF-IDF - Linear SVC',
             3 : 'Glove 300d - TF-IDF - Random Forest',
             4 : 'Doc2Vec - Linear SVC',
             5 : 'Doc2Vec - Random Forest',
             6 : 'All Features - Linear SVC',
             7 : 'All Features - Random Forest',}

In [64]:
from sklearn import svm
from sklearn.metrics import make_scorer, mean_absolute_error

pip_local_w2v_tfidf_naive = Pipeline([
    ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

pip_glove_tfidf_naive = Pipeline([
    ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

pip_glove_tfidf_rf = Pipeline([
    ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

# doc2vec_svc = Pipeline([
#     ('doc2vec', Doc2VecTransformer()),
#     ('clf', MultiOutputClassifier(svm.LinearSVC(random_state=199)))
# ])

# all_feats_svc = Pipeline([
#     ('features', FeatureUnion([
#         ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
#         ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
#         ('doc2vec', Doc2VecTransformer()),
#         ('cats_sim', CategoriesSimilarity(glove_300d_w2v, categories_tokens))
#     ])),
#     ('clf', MultiOutputClassifier(svm.LinearSVC(random_state=199)))
# ])

grid_params_empty = [{}]

# Construct grid searches
jobs = -1

score = make_scorer(mean_absolute_error, greater_is_better=False)

gs_local_w2v_tfidf_naive = GridSearchCV(estimator=pip_local_w2v_tfidf_naive,
            param_grid=grid_params_empty,
            scoring=score,
            cv=2) 

gs_glove_tfidf_rf = GridSearchCV(estimator=pip_glove_tfidf_rf,
            param_grid=grid_params_empty,
            scoring=score,
            cv=2,
            n_jobs=jobs) 

gs_glove_tfidf_naive = GridSearchCV(estimator=pip_glove_tfidf_naive,
            param_grid=grid_params_empty,
            scoring=score,
            cv=2,
            n_jobs=jobs) 

grids = [gs_glove_tfidf_naive]

grid_dict = {
    0 : 'Glove 300d - TF-IDF - Naive Bayes',
    1 : 'Local Word2Vec - TF-IDF - Naive Bayes'
}

In [69]:
%%time

print('Performing model optimizations...')
best_score = 0.0
best_clf = 0
best_gs = ''
overall_results_df = pd.DataFrame()
for idx, gs in enumerate(grids):
    print('\nEstimator: %s' % grid_dict[idx])
    # Fit grid search
    gs.fit(X_train, y_train)
    # Best params
    print('Best params: %s' % gs.best_params_)
    # Best training data accuracy
    print('Best training accuracy: %.3f' % gs.best_score_)
    # Predict on test data with best params
    test_score = gs.score(X_test, y_test)
    # Test data accuracy of model with best params
    print('Test set accuracy score for best params: %.3f ' % test_score)
    
    # Track Grid Search results
    gs_results_df = pd.DataFrame(gs.cv_results_)
    gs_results_df['grid_id'] = grid_dict[idx]
    gs_results_df['best_model_test_score'] = test_score
    
    overall_results_df = pd.concat([overall_results_df, gs_results_df])
    
    # Track best (highest test accuracy) model
    if test_score > best_score:
        best_score = test_score
        best_gs = gs
        best_clf = idx
print('\nClassifier with best test set accuracy: %s' % grid_dict[best_clf])

Performing model optimizations...

Estimator: Glove 300d - TF-IDF - Naive Bayes
Best params: {}
Best training accuracy: -0.609
Test set accuracy score for best params: -0.623 

Classifier with best test set accuracy: Glove 300d - TF-IDF - Naive Bayes
CPU times: user 1min 26s, sys: 2.27 s, total: 1min 28s
Wall time: 2min 55s


In [70]:
overall_results_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score,grid_id,best_model_test_score
0,39.863238,2.186177,47.271844,3.999239,{},-0.59746,-0.620781,-0.609121,0.01166,1,Glove 300d - TF-IDF - Naive Bayes,-0.623321


# Save best grid search pipeline to file
dump_file = 'best_gs_pipeline.pkl'
joblib.dump(best_gs, dump_file, compress=1)
print('\nSaved %s grid search pipeline to file: %s' % (grid_dict[best_clf], dump_file))

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

X_train.shape

X_train

# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

print(X_test.shape)

print(y_pred.shape)
y_pred

y_pred[:,0]

y_test

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[df.columns[4+category_idx] + '-0',df.columns[4+category_idx] + '-1']))

### 6. Improve your model
Use grid search to find better parameters. 

pipeline.get_params().keys()

parameters = {
        'multi_clf__estimator__n_estimators': [20,50],
        'multi_clf__estimator__max_depth': [50, 100]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

# train classifier
cv.fit(X_train, y_train)

# predict on test data
y_pred = cv.predict(X_test)

cv.best_params_

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[df.columns[4+category_idx] + '-0',df.columns[4+category_idx] + '-1']))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

# Output a pickle file for the model
joblib.dump(cv, 'classifier.pkl') 

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.